From raw rec.gov data, transform files to final working form for understanding max capacity of rec facilities that are for camping.

In [1]:
from dask import delayed
from dask import compute

In [2]:
import swifter

In [3]:
import boto3
import os
import pandas as pd
import dask.dataframe as dd

set up dask client

In [2]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:38797 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 18.68 GiB


set up the aws connection

In [4]:
s3 = boto3.client("s3")

bucket = 'rec-gov-study'

upload the raw data to an s3 bucket

In [5]:
%%time
Campsites_API_file = '../data/Campsites_API_v1.csv'
#s3.upload_file(Campsites_API_file, bucket, Campsites_API_file)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.77 µs


In [6]:
%%time
fy20_historical_reservations_file = '../data/fy20_historical_reservations_full.csv'
#s3.upload_file(fy20_historical_reservations_file, bucket, fy20_historical_reservations_file)

CPU times: user 0 ns, sys: 2 µs, total: 2 µs
Wall time: 5.25 µs


considering how long it takes to retrieve the full raw data from s3, work locally to create the main files to work with

In [ ]:
%%time
boto_object = s3.get_object(Bucket=bucket, Key=file)
pd.read_csv(boto_object['Body'], nrows=300000)

load files and convert the dates to the correct data type (coercing when necessary)

In [7]:
date_cols = ['startdate', 'enddate', 'orderdate']

df_all_res = pd.read_csv(fy20_historical_reservations_file, parse_dates=date_cols)
df_campsite_api = pd.read_csv(Campsites_API_file)

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,6,7,11,12,13,15,17,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df_all_res['enddate'] = pd.to_datetime(df_all_res['enddate'], errors='coerce')

In [9]:
df_all_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5114789 entries, 0 to 5114788
Data columns (total 37 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   historicalreservationid  object        
 1   ordernumber              object        
 2   agency                   object        
 3   orgid                    int64         
 4   codehierarchy            object        
 5   regioncode               object        
 6   regiondescription        object        
 7   parentlocationid         object        
 8   parentlocation           object        
 9   legacyfacilityid         float64       
 10  park                     object        
 11  sitetype                 object        
 12  usetype                  object        
 13  productid                object        
 14  inventorytype            object        
 15  facilityid               object        
 16  facilityzip              float64       
 17  facilitystate            ob

In [10]:
df_all_res.columns

Index(['historicalreservationid', 'ordernumber', 'agency', 'orgid',
       'codehierarchy', 'regioncode', 'regiondescription', 'parentlocationid',
       'parentlocation', 'legacyfacilityid', 'park', 'sitetype', 'usetype',
       'productid', 'inventorytype', 'facilityid', 'facilityzip',
       'facilitystate', 'facilitylongitude', 'facilitylatitude', 'customerzip',
       'customerstate', 'customercountry', 'tax', 'usefee', 'tranfee',
       'attrfee', 'totalbeforetax', 'discount', 'totalpaid', 'startdate',
       'enddate', 'orderdate', 'nights', 'numberofpeople',
       'equipmentdescription', 'equipmentlength'],
      dtype='object')

looking at the campsite api, how many campsites are at each facility

In [11]:
df_facility_max = df_campsite_api.groupby('FacilityID')['CampsiteID'].count().reset_index()
df_facility_max.columns = ['facilityid', 'total_num_campsites']

In [12]:
df_facility_max.head()

,facilityid,total_num_campsites
0,72417,15
1,72481,1
2,118290,15
3,118440,28
4,118990,1


Identify what reservations are campsite related based on facility ID?

Could facility ID indicate more than just a campsite (does this join need to be done on facility id and sitetype?)

In [13]:
df_merge_all_data = pd.merge(df_all_res, df_facility_max, on='facilityid', how='left')

identify the number of reservations associated with campsites with the reservation records

In [14]:
df_merge_all_data['campsite'] = ~df_merge_all_data['total_num_campsites'].isnull()

In [15]:
df_merge_all_data.head()

,historicalreservationid,ordernumber,agency,orgid,codehierarchy,regioncode,regiondescription,parentlocationid,parentlocation,legacyfacilityid,...,totalpaid,startdate,enddate,orderdate,nights,numberofpeople,equipmentdescription,equipmentlength,total_num_campsites,campsite
0,fa50401c-f08a-4738-b576-56335cc19cb7,0121072064-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195,White Mountains National Recreation Area,146090.0,...,33.0,2019-10-17,2019-10-18,2019-09-18 16:25:42.280,1 day,6.0,NaN,0.0,14.0,True
1,09c37303-cf15-49d9-9b0c-ccc120328bbb,0200063847-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195,White Mountains National Recreation Area,146090.0,...,33.0,2020-04-11,2020-04-12,2020-03-13 16:35:11.612,1 day,1.0,NaN,0.0,14.0,True
2,f1f5c841-46b1-4ff8-a6dc-fcac2e76ae1c,0200087454-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195,White Mountains National Recreation Area,146090.0,...,83.0,2020-09-07,2020-09-10,2020-08-17 05:19:08.159,3 days,2.0,NaN,0.0,14.0,True
3,619f4394-1cac-4a64-8f0c-eae6e7f6bf3b,0200102788-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195,White Mountains National Recreation Area,146090.0,...,33.0,2020-03-21,2020-03-22,2020-02-20 15:01:33.904,1 day,3.0,NaN,0.0,14.0,True
4,82674243-7f31-41e6-b3db-d0690513cad3,0200147542-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195,White Mountains National Recreation Area,146090.0,...,0.0,2019-12-07,2019-12-08,2019-11-07 17:54:01.448,1 day,2.0,NaN,0.0,14.0,True


create a dataframe of only camping reservations

select the unique for facililty id, product id, start date, end date, take most recent order date

In [16]:
df_merge_all_data_camping = df_merge_all_data[df_merge_all_data['campsite']][['facilityid', 'productid', 'startdate', 'enddate', 'orderdate']]

In [17]:
df_merge_all_data_camping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3270375 entries, 0 to 5107392
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   facilityid  object        
 1   productid   object        
 2   startdate   datetime64[ns]
 3   enddate     datetime64[ns]
 4   orderdate   datetime64[ns]
dtypes: datetime64[ns](3), object(2)
memory usage: 149.7+ MB


In [18]:
%%time
df_merge_all_data_camping_unique = df_merge_all_data_camping.groupby(['facilityid', 'productid', 'startdate', 'enddate']).max('orderdate')

CPU times: user 5min 11s, sys: 332 ms, total: 5min 11s
Wall time: 5min 11s


In [19]:
df_merge_all_data_camping_unique.reset_index(inplace=True)

In [20]:
df_merge_all_data_camping_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086689 entries, 0 to 3086688
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   facilityid  int64         
 1   productid   int64         
 2   startdate   datetime64[ns]
 3   enddate     datetime64[ns]
 4   orderdate   datetime64[ns]
dtypes: datetime64[ns](3), int64(2)
memory usage: 117.7 MB


In [22]:
file = '../data/df_merge_all_data_camping_unique_facil_start_end.csv'
s3_file = 'data/df_merge_all_data_camping_unique_facil_start_end.csv'
df_merge_all_data_camping_unique.to_csv(file)

s3.upload_file(file, bucket, s3_file)

For each record calculate if it was a likely cancelation based on if the product id was booked again at a later date for the same start range (or any part of that date range)

the following does not work because it is too big of a dataset. Ran for over a day without output. 

In [91]:

def reservations_likely_canceled(row):
    '''if there is another reservation made at a later date that has the same facility and 
    productuctid and is within the same reservation date block (between start and end) then 
    consider this one to be a likely canceled one.'''
    
    
    facil = row['facilityid']
    campsite = row['productid']
    sdate = row['startdate']
    edate = row['enddate']
    odate = row['orderdate']
    
    
    #select all rows where the start date is after and facil and campsite number are the same
    #(StartA <= EndB) and (EndA >= StartB)
    df_other_res = df_merge_all_data_camping_unique[(df_merge_all_data_camping_unique['facilityid']==facil) & 
                  (df_merge_all_data_camping_unique['productid']==campsite) & 
                  (df_merge_all_data_camping_unique['startdate']<=edate) &
                  (df_merge_all_data_camping_unique['enddate']>=sdate) &
                  (df_merge_all_data_camping_unique['orderdate']>odate)]
    
    if df_other_res.empty:
        return False
    else:
        return True
    


In [92]:
%%time
df_merge_all_data_camping_unique['cancelation_likely'] = df_merge_all_data_camping_unique.swifter.apply(reservations_likely_canceled, axis=1)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [29]:
df_merge_all_data.head()

,historicalreservationid,ordernumber,agency,orgid,codehierarchy,regioncode,regiondescription,parentlocationid,parentlocation,legacyfacilityid,...,startdate,enddate,orderdate,nights,numberofpeople,equipmentdescription,equipmentlength,total_num_campsites,campsite,cancelation_likely
0,fa50401c-f08a-4738-b576-56335cc19cb7,0121072064-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195.0,White Mountains National Recreation Area,146090.0,...,2019-10-17,2019-10-18,2019-09-18 16:25:42.280,1 day,6.0,NaN,0.0,14.0,True,True
1,09c37303-cf15-49d9-9b0c-ccc120328bbb,0200063847-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195.0,White Mountains National Recreation Area,146090.0,...,2020-04-11,2020-04-12,2020-03-13 16:35:11.612,1 day,1.0,NaN,0.0,14.0,True,True
2,f1f5c841-46b1-4ff8-a6dc-fcac2e76ae1c,0200087454-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195.0,White Mountains National Recreation Area,146090.0,...,2020-09-07,2020-09-10,2020-08-17 05:19:08.159,3 days,2.0,NaN,0.0,14.0,True,True
3,619f4394-1cac-4a64-8f0c-eae6e7f6bf3b,0200102788-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195.0,White Mountains National Recreation Area,146090.0,...,2020-03-21,2020-03-22,2020-02-20 15:01:33.904,1 day,3.0,NaN,0.0,14.0,True,True
4,82674243-7f31-41e6-b3db-d0690513cad3,0200147542-1,BLM,126,|1|70901|72193|72194|146090,AK,Alaska (BLM),72195.0,White Mountains National Recreation Area,146090.0,...,2019-12-07,2019-12-08,2019-11-07 17:54:01.448,1 day,2.0,NaN,0.0,14.0,True,False


In [37]:

def reservations_likely_canceled(row):
    '''if there is another reservation made at a later date that has the same facility and 
    productuctid and is within the same reservation date block (between start and end) then 
    consider this one to be a likely canceled one.'''
    
    
    facil = row['facilityid']
    campsite = row['productid']
    sdate = row['startdate']
    edate = row['enddate']
    odate = row['orderdate']
    
    
    #select all rows where the start date is after and facil and campsite number are the same
    #(StartA <= EndB) and (EndA >= StartB)
    df_other_res = df_merge_all_data[(df_merge_all_data['facilityid']==facil) & 
                  (df_merge_all_data['productid']==campsite) & 
                  (df_merge_all_data['startdate']<=edate) &
                  (df_merge_all_data['enddate']>=sdate) &
                  (df_merge_all_data['orderdate']>odate)]
    
    if df_other_res.empty:
        return False
    else:
        return True
    


In [38]:
%%time
df_merge_all_data['cancelation_likely'] = df_merge_all_data.swifter.apply(reservations_likely_canceled, axis=1)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

CPU times: user 17.1 s, sys: 2.28 s, total: 19.4 s
Wall time: 2min 7s


Attempt a similar process but using dask

In [16]:
ddf = dd.from_pandas(df_merge_all_data, npartitions=4)

In [37]:
def reservations_likely_canceled_ddf(row):
    '''if there is another reservation made at a later date that has the same facility and 
    productuctid and is within the same reservation date block (between start and end) then 
    consider this one to be a likely canceled one.'''
    
    
    facil = row['facilityid']
    campsite = row['productid']
    sdate = row['startdate']
    edate = row['enddate']
    odate = row['orderdate']
    
    
    #select all rows where the start date is after and facil and campsite number are the same
    #(StartA <= EndB) and (EndA >= StartB)
    df_other_res = ddf[(ddf['facilityid']==facil) & 
                  (ddf['productid']==campsite) & 
                  (ddf['startdate']<=edate) &
                  (ddf['enddate']>=sdate) &
                  (ddf['orderdate']>odate)]
    
    if len(df_other_res.index) == 0:
        return False
    else:
        return True

In [ ]:
%%time
ddf['cancelation_likely'] = ddf.map_partitions(reservations_likely_canceled_ddf, meta=(None, 'boolean')).compute()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [17]:
%%time
facil = 232445
campsite = 371
sdate = '2020-09-18 00:00:00'
edate = '2020-09-20 00:00:00'

df_merge_all_data[(df_merge_all_data['facilityid']==facil) & 
                  (df_merge_all_data['productid']==campsite) & 
                  (df_merge_all_data['startdate']<edate) &
                     (df_merge_all_data['enddate']>sdate)]

CPU times: user 2.16 s, sys: 1.51 s, total: 3.67 s
Wall time: 3.66 s


,historicalreservationid,ordernumber,agency,orgid,codehierarchy,regioncode,regiondescription,parentlocationid,parentlocation,legacyfacilityid,...,totalpaid,startdate,enddate,orderdate,nights,numberofpeople,equipmentdescription,equipmentlength,total_num_campsites,campsite
472188,753e51bc-c714-4026-90d9-8f5921e870dc,0208169760-1,NPS,128,|1|70904|74324|74297|70923,RMR,Intermountain Region,74297,Zion National Park,70923.0,...,10.0,2020-09-17,2020-09-21,2020-03-21 19:45:37.770,4 days,6.0,Tent,0.0,188.0,True
475822,c23dd8c6-8190-403b-9aea-713565b3f459,0218642418-1,NPS,128,|1|70904|74324|74297|70923,RMR,Intermountain Region,74297,Zion National Park,70923.0,...,40.0,2020-09-18,2020-09-20,2020-09-11 16:34:11.903,2 days,4.0,Tent,0.0,188.0,True
485505,4c216018-339a-4bde-a14a-ae9f469e722a,0246310809-1,NPS,128,|1|70904|74324|74297|70923,RMR,Intermountain Region,74297,Zion National Park,70923.0,...,10.0,2020-09-18,2020-09-20,2020-08-27 18:50:52.941,2 days,4.0,Tent,0.0,188.0,True
498200,aaf65416-99de-40ae-b3c6-11521885e85f,0282784799-1,NPS,128,|1|70904|74324|74297|70923,RMR,Intermountain Region,74297,Zion National Park,70923.0,...,10.0,2020-09-18,2020-09-20,2020-08-17 18:02:47.100,2 days,2.0,Tent,0.0,188.0,True


In [32]:
%%time
facil = 232445
campsite = 371
sdate = '2020-09-18 00:00:00'
edate = '2020-09-20 00:00:00'

ddf[(ddf['facilityid']==facil) & 
                  (ddf['productid']==campsite) & 
                  (ddf['startdate']<edate) &
                     (ddf['enddate']>sdate)]['ordernumber']

CPU times: user 4.85 ms, sys: 546 µs, total: 5.39 ms
Wall time: 5.05 ms


Dask Series Structure:
npartitions=4
0          object
1278698       ...
2557396       ...
3836094       ...
5114788       ...
Name: ordernumber, dtype: object
Dask Name: getitem, 56 tasks

looking at the reservation records, identify the unique facilityid, facility name, parentlocation, and parentlocationid for the purpose of joining back to df_facility_max

Before I do this, am I sure that this does not exist in another table? - there is a facility API table that has parent location id infomormation but not parent location name.

In [ ]:
df_merge_all_data.columns

In [ ]:
df_merge_all_data[['facilityid', 'park', 'parentlocationid', 'parentlocation', 'productid']]

select from the resulting reservation table only the columns of interest

In [ ]:
df_merge_all_data.info()